In [18]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd

In [19]:
path='‪D:/22monidata/wrfout_d02_2017-01-01_00_00_00'
raw_data=Dataset(path)

In [20]:
nx = pd.Series(np.arange(1,91)) #nx东西格点数
ny = pd.Series(np.arange(1,94)) #ny南北格点数
nl = pd.Series(np.arange(1,86))
nt = pd.Series(np.arange(1,26))

In [21]:
import itertools
# index的笛卡尔乘积。注意：高维在前，低维在后
prod = itertools.product(nt,nx,ny,nl)
# 转换为DataFrame
prod = pd.DataFrame([x for x in prod])
prod.columns = [ 'nt','nx','ny', 'nl']
prod.T

,0,1,2,3,4,5,6,7,8,9,...,17786240,17786241,17786242,17786243,17786244,17786245,17786246,17786247,17786248,17786249
nt,1,1,1,1,1,1,1,1,1,1,...,25,25,25,25,25,25,25,25,25,25
nx,1,1,1,1,1,1,1,1,1,1,...,90,90,90,90,90,90,90,90,90,90
ny,1,1,1,1,1,1,1,1,1,1,...,93,93,93,93,93,93,93,93,93,93
nl,1,2,3,4,5,6,7,8,9,10,...,76,77,78,79,80,81,82,83,84,85


In [22]:
raw_data.variables['T'][:]    # 访问变量数据的方法，返回numpy掩码数组
np.array(raw_data.variables['T'][:])     # 可以直接转成numpy数组
# 利用‘P’和‘PB’计算气压值pressure
pressure = (raw_data.variables['P'][:] + raw_data.variables['PB'][:]) / 100
pressure.shape

(25, 85, 93, 90)

In [23]:
# 利用‘PH’和‘PHB’计算高度值height
PH, PHB = raw_data.variables['PH'][:], raw_data.variables['PHB'][:]
PHH = 0.5 * (PH[:, :85, :, :] + PH[:, 1:86, :, :])
PHBH = 0.5 * (PHB[:, :85, :, :] + PHB[:, 1:86, :, :])
GEOPT = PHH + PHBH
height = GEOPT / 9.81   # 求高度值
height.shape

(25, 85, 93, 90)

In [24]:
# 利用‘T’和‘pressure’计算温度值T
T = raw_data.variables['T'][:]
Tem = (T + 300.) / (1000. / pressure) **0.286
Tem.shape

(25, 85, 93, 90)

In [25]:
# 利用‘QVAPOR’计算水气压值
QVAPOR = raw_data.variables['QVAPOR'][:]  # 水汽混合比
e = QVAPOR * pressure / (0.622 + QVAPOR) 
QVAPOR.shape,e.shape

((25, 85, 93, 90), (25, 85, 93, 90))

In [26]:
XLONG = raw_data.variables['XLONG'][:]    # 读取经度值
HGT = raw_data.variables['HGT'][:]    # 读取海拔高度
XLAT = raw_data.variables['XLAT'][:]    # 读取纬度值
XLONG.shape,XLAT.shape,HGT.shape

((25, 93, 90), (25, 93, 90), (25, 93, 90))

In [27]:
xlong, xlat,hgt = np.zeros((25,93,90,85)), np.zeros((25,93,90,85)), np.zeros((25,93,90,85))
#转化为四维数组
for i in range(85):
    xlong[:,:,:,i]=XLONG[:,:,:]
    xlat[:,:,:,i]=XLAT[:,:,:]
    hgt[:,:,:,i]=HGT[:,:,:]
xlong.shape,xlat.shape,hgt.shape

((25, 93, 90, 85), (25, 93, 90, 85), (25, 93, 90, 85))

In [28]:
# 根据Fortran的代码转化数据维度  (nx东西格点数, ny南北格点数, nl垂直层数, nt时次数)
e = e.transpose((0,3, 2, 1))
Tem = Tem.transpose((0,3, 2, 1))
QVAPOR = QVAPOR.transpose((0,3, 2, 1))
pressure = pressure.transpose((0,3, 2, 1))
height = height.transpose((0,3, 2, 1))
xlong = xlong.transpose((0,2, 1, 3))
hgt = hgt.transpose((0,2, 1, 3))
xlat = xlat.transpose((0,2, 1, 3))
z=height-hgt
[e.shape, Tem.shape, pressure.shape, height.shape, hgt.shape, xlat.shape ,xlong.shape,z.shape]

[(25, 90, 93, 85),
 (25, 90, 93, 85),
 (25, 90, 93, 85),
 (25, 90, 93, 85),
 (25, 90, 93, 85),
 (25, 90, 93, 85),
 (25, 90, 93, 85),
 (25, 90, 93, 85)]

In [29]:
# 计算折射率，NN是原始折射率，MM是修正折射率
re = 6371.
NN = (3.73*10**5*e)/(Tem**2)+(77.6*pressure)/(Tem)
MM = (3.73*10**5*e)/(Tem**2)+(77.6*pressure)/(Tem)+((height-hgt)/(re*1000.)*10**6)
NN.shape, MM.shape

((25, 90, 93, 85), (25, 90, 93, 85))

In [30]:
a=xlong.flatten()
a = pd.Series(a)
a.name = 'xlong'

b=xlat.flatten()
b = pd.Series(b)
b.name = 'xlat'

c=hgt.flatten()
c = pd.Series(c)
c.name = 'hgt'

o=height.flatten()
o=pd.Series(o)
o.name='height'

d=z.flatten()
d = pd.Series(d)
d.name = 'z'

f=e.flatten()
f = pd.Series(f)
f.name = 'e'

g=pressure.flatten()
g = pd.Series(g)
g.name = 'pressure'

h=Tem.flatten()
h = pd.Series(h)
h.name = 'Tem'

l=QVAPOR.flatten()
l = pd.Series(l)
l.name = 'QVAPOR'

mm=MM.flatten()
mm = pd.Series(mm)
mm.name = 'MM'

nn=NN.flatten()
nn = pd.Series(nn)
nn.name = 'NN'

In [31]:
#最终数据，合并成一个DataFrame
data=pd.concat([prod,a,b,c,o,d,f,g,h,l,mm,nn],axis=1)
data1=pd.DataFrame(data)
data2=data1.iloc[0:711450,:]
data3=data2.loc[data2['nl']<46,:]
data2,data3

(        nt  nx  ny  nl       xlong       xlat        hgt        height  \
 0        1   1   1   1  104.101074   1.387764  18.679893     21.906155   
 1        1   1   1   2  104.101074   1.387764  18.679893     27.956425   
 2        1   1   1   3  104.101074   1.387764  18.679893     32.395199   
 3        1   1   1   4  104.101074   1.387764  18.679893     36.432667   
 4        1   1   1   5  104.101074   1.387764  18.679893     40.874550   
 ...     ..  ..  ..  ..         ...        ...        ...           ...   
 711445   1  90  93  81  126.461487  22.871384   0.000000  14474.018555   
 711446   1  90  93  82  126.461487  22.871384   0.000000  15297.822266   
 711447   1  90  93  83  126.461487  22.871384   0.000000  15847.804688   
 711448   1  90  93  84  126.461487  22.871384   0.000000  16040.171875   
 711449   1  90  93  85  126.461487  22.871384   0.000000  16362.005859   
 
                    z          e     pressure         Tem    QVAPOR  \
 0           3.226261  29.1

In [32]:
#改动时间和日期
data2.to_csv("D:/20年海洋/数据/模拟数据/层数层面导出数据/20170101nt1.csv" , encoding = "utf-8", header=True, index=None)

In [33]:
#改动时间和日期
data3.to_csv("D:/20年海洋/数据/模拟数据/3000m层数层面导出数据/20170101nt1.csv" , encoding = "utf-8", header=True, index=None)